## Drug Review Summarization Implementation

- Huggingface Transformer
- Note: Gemma-2b-it is a gated Model (Huggingface Access Token Required)
- Pre-trained Model : google/gemma-2b-it
- Prompt Engingeering

In [4]:
# !pip uninstall transformers
# !pip install transformers

In [5]:
import torch

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available")
else:
    device = torch.device("cpu")
    print("CPU used")

GPU is available


In [7]:
from huggingface_hub import login
login()

In [8]:
# ! pip install -U accelerate
# ! pip install -U transformers

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [10]:
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# model = AutoModelForCausalLM.from_pretrained(
#     "google/gemma-2b-it",
#     device_map="auto",
#     torch_dtype=torch.bfloat16
# )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [23]:
#moving to gpu
model.to(device)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

In [24]:
# model = GemmaModel.from_pretrained("google/gemma-2b-it")
# tokenizer = GemmaTokenizer.from_pretrained("google/gemma-2b-it")

Review to explain:

Restoril helps me get to sleep and helps prevent me from waking up constantly during the night.
I have had no side-effects from Restoril.
Document id: 7
Probability(positive) = 0.7269413
True class: positive

In [25]:
LIME_Output = [('no', 0.12353220934498445),
 ('helps', 0.12086655765071426),
 ('had', -0.07319883184685684),
 ('Restoril', 0.06043515711583609),
 ('night', 0.049670059792065006),
 ('sleep', 0.036615041037947105),
 ('effects', 0.03476128305091525),
 ('during', -0.027915043303471526),
 ('me', -0.027467910992129266),
 ('to', -0.027282000344150334)]

drug = ['Restoril']

condition = ['insomnia']

In [26]:
# LIME_indicator_words = [word for word, _ in LIME_Output]
# print(LIME_indicator_words)

In [27]:
# word_weight_dict = {word: weight for word, weight in LIME_Output}
# print(word_weight_dict)

In [28]:
pos_words = [word for word, weight in LIME_Output if weight > 0]
neg_words = [word for word, weight in LIME_Output if weight < 0]

In [29]:
def inputVariables(pos_words,neg_words, drug, condition):
  return drug, condition, pos_words, neg_words

In [30]:
drug, condition, pos_words, neg_words = inputVariables(pos_words,neg_words, drug, condition)
print(drug)
print(condition)
print(pos_words)
print(neg_words)

['Restoril']
['insomnia']
['no', 'helps', 'Restoril', 'night', 'sleep', 'effects']
['had', 'during', 'me', 'to']


In [60]:
def promptTemplate(inputVariables):
    print(inputVariables[0][0])
    print(inputVariables[1])
    print(inputVariables[2])
    final_prompt = (
            f"Generate a summary of text not bullet points of patient medicine review for the drug {inputVariables[0]} which is used to treat"
            f"{inputVariables[1]} based on the words {inputVariables[2]} determined to be most positively correlated "
            f"with sentiment classifier and these words {inputVariables[3]} determined to be most correlated with "
            f"sentiment classifier negative output.Ensure that the negatve outpus and postivie output words clearly show negative and positive sentiment respectively."
            f"do not provide any extra explanation outside of patient sentiment ")

    return final_prompt


In [63]:
input_text = promptTemplate(inputVariables(pos_words,neg_words, drug, condition))

Restoril
['insomnia']
['no', 'helps', 'Restoril', 'night', 'sleep', 'effects']


In [76]:
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=200)

# generated_sequence = outputs[0][len(input_ids):]
print(tokenizer.decode(outputs[0]))

<bos>Generate a summary of text not bullet points of patient medicine review for the drug ['Restoril'] which is used to treat['insomnia'] based on the words ['no', 'helps', 'Restoril', 'night', 'sleep', 'effects'] determined to be most positively correlated with sentiment classifier and these words ['had', 'during', 'me', 'to'] determined to be most correlated with sentiment classifier negative output.Ensure that the negatve outpus and postivie output words clearly show negative and positive sentiment respectively.do not provide any extra explanation outside of patient sentiment 

Restoril helps patients to fall asleep during the night and sleep through the day. It is a safe and effective treatment for insomnia. However, some patients may experience side effects, such as drowsiness and fatigue. Restoril can also have a negative impact on sleep if taken too close to bedtime.<eos>


In [78]:
import pprint

In [79]:
pprint.pp(tokenizer.decode(outputs[0]))

('<bos>Generate a summary of text not bullet points of patient medicine review '
 "for the drug ['Restoril'] which is used to treat['insomnia'] based on the "
 "words ['no', 'helps', 'Restoril', 'night', 'sleep', 'effects'] determined to "
 'be most positively correlated with sentiment classifier and these words '
 "['had', 'during', 'me', 'to'] determined to be most correlated with "
 'sentiment classifier negative output.Ensure that the negatve outpus and '
 'postivie output words clearly show negative and positive sentiment '
 'respectively.do not provide any extra explanation outside of patient '
 'sentiment \n'
 '\n'
 'Restoril helps patients to fall asleep during the night and sleep through '
 'the day. It is a safe and effective treatment for insomnia. However, some '
 'patients may experience side effects, such as drowsiness and fatigue. '
 'Restoril can also have a negative impact on sleep if taken too close to '
 'bedtime.<eos>')
